# Lab 7-1: Tips

Author: Seungjae Lee (이승재)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# For reproducibility
torch.manual_seed(1)

## Training and Test Datasets

In [ ]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [ ]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

## Model

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1)#이 부분을 채워넣으세요, #이 부분을 채워넣으세요) #x_train의 shape과 y_train의 class의 갯수를 고려해보세요.
    def forward(self, x):
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel()#이 부분을 채워넣으세요.# #SoftmaxClassifierModel()로 모델을 초기화하세요.

In [ ]:
# optimizer 설정
 #이 부분을 채워넣으세요.# #SGD optimizer를 사용하고 learning rate는 0.1로 적용하세요.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)#이 부분을 채워넣으세요.# #모델을 생성해서 예측값을 구해보세요.

        # cost 계산
        cost = F.cross_entropy(prediction,y_train)#이 부분을 채워넣으세요.# #PyTorch에서 제공하는 F.cross_entropy함수를 사용해 cost를 구현해보세요.

        # cost로 H(x) 개선
        optimizer.zero_grad() #이 부분을 채워넣으세요.#  # gradient를 0으로 초기화
        cost.backward()#이 부분을 채워넣으세요.#  # 비용 함수를 미분하여 gradient 계산
        optimizer.step()#이 부분을 채워넣으세요.#  # W와 b를 업데이트

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [ ]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1] # 각각의 데이터 마다 가장 큰 값의 인덱스를 취함 => class를 예측하는 것
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [ ]:
train(model, optimizer, x_train, y_train)

RuntimeError: ignored

In [ ]:
test(model, optimizer, x_test, y_test)

IndexError: ignored

## Learning Rate

Gradient Descent 에서의 $\alpha$ 값

`optimizer = optim.SGD(model.parameters(), lr=0.1)` 에서 `lr=0.1` 이다

Q1. Learning Rate가 다음과 같을 때 어떤 일이 발생할까요?

Learing rate가 클 경우 :

Learing rate가 작을 경우 : 

Large learning rate

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [ ]:
train(model, optimizer, x_train, y_train)

TypeError: ignored

Small learning rate

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [ ]:
train(model, optimizer, x_train, y_train)

TypeError: ignored

적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
train(model, optimizer, x_train, y_train)

TypeError: ignored

## Data Preprocessing (데이터 전처리)

데이터를 zero-center하고 normalize하자.

In [ ]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$ x'_j = \frac{x_j - \mu_j}{\sigma_j} $$

여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다.

In [ ]:
mu = x_train.mean(dim=0)

In [ ]:
sigma = x_train.std(dim=0)

In [ ]:
norm_x_train = (x_train - mu) / sigma

In [ ]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


Normalize와 zero center한 X로 학습해서 성능을 보자

In [ ]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
model = MultivariateLinearRegressionModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [ ]:
train(model, optimizer, #전처리한 데이터를 사용해보세요!, y_train)

SyntaxError: ignored

Q2. 이렇게 Data Preprocessing을 하는 이유가 무엇일까요?

=>

## Overfitting

Q3. Overfitting의 개념은?

=> 

Q4. 그렇다면 Overfitting을 방지하는 방법에는 무엇이 있을까요?

=> 

Regularization: Let's not have too big numbers in the weights

In [ ]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        cost += l2_reg

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

In [ ]:
model = MultivariateLinearRegressionModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
train_with_regularization(model, optimizer, norm_x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if __name__ == '__main__':


RuntimeError: ignored